In [1]:
import tequila as tq
import numpy as np
from numpy import pi
import qiskit

In [2]:
from qiskit import IBMQ
MY_IBMQ_TOKEN = '55dee9b655b84737468de95877d2c72ecf8d7fc6b40cf6edb2b96668fbe80b40381e0f216fd0ab70a256ac97ad4030cef9ab39f3aae616720069fa2c5b9a111c'
IBMQ.save_account(MY_IBMQ_TOKEN, overwrite=True) # needs to be done once, and is then saved in our environment
IBMQ.load_account()
# see the list of backends that you can use:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]


In [4]:
def fSWAP(q0,q1):
    circuit = tq.gates.CNOT(q0,q1) + tq.gates.CNOT(q1,q0) + tq.gates.CNOT(q0,q1)
    circuit += tq.gates.CZ(q0,q1)
    return circuit
    

In [5]:
# CH (Controlled-Haddamard)
# control qubit: q1
# target qubit: q0
def CH2(q0,q1):
    circuit = tq.gates.Phase(phi=-pi/2,target=q0) + tq.gates.H(q0) + tq.gates.Phase(phi=-pi/4,target=q0)
    circuit += tq.gates.H(q0) + tq.gates.H(q1) + tq.gates.CNOT(q0,q1) + tq.gates.H(q0) +tq.gates.H(q1)
    circuit += tq.gates.T(q0) + tq.gates.H(q0) + tq.gates.S(q0)
    return circuit
qiskit_mix = tq.compile(CH2(0,1),backend = 'qiskit')
# Fourier transform gates

def F2(q0,q1):
    circuit = tq.gates.CNOT(q0,q1) + CH2(q0,q1) + tq.gates.CNOT(q0,q1) +tq.gates.CZ(q0,q1)
    return circuit

def F0(q0,q1):
    return F2(q0,q1)
    
def F1(q0,q1):
    return F2(q0,q1) + tq.gates.Phase(phi=-pi/2,target=q0)


In [6]:
from math import pi

# control qubit: q0
# target qubit: q1
def CRX(theta, q0, q1):
    circuit = tq.gates.Rz(angle=pi/2.0,target=q1) + tq.gates.Ry(angle=theta, target=q1) + tq.gates.CNOT(q0,q1)
    circuit += tq.gates.Ry(angle=-pi/2.0,target=q1) + tq.gates.CNOT(q0,q1) + tq.gates.Rz(angle=-pi/2.0,target=q1)
    return circuit

def B(theta_k,q0,q1):
#     circuit = tq.gates.X(q1) + tq.gates.CX(control=q1,target=q0) +CRX(theta_k,q0,q1) + tq.gates.CX(control=q1,target=q0)+tq.gates.X(q1)
    circuit = tq.gates.X(q1) + tq.gates.CX(control=q1,target=q0) +tq.gates.CRx(control=q0,target=q1,angle=theta_k) + tq.gates.CX(control=q1,target=q0)+tq.gates.X(q1)

    return circuit

In [7]:
def Udisg(lam,q0,q1,q2,q3):
    k=1
    n=4
    th1=-np.arccos((lam-np.cos(2*pi*k/n))/np.sqrt((lam-np.cos(2*pi*k/n))**2+np.sin(2*pi*k/n)**2))
    circuit = B(th1,q0,q1)+ F1(q0,q1) + F0(q2,q3)
    #fSWAP(Udis,q2,q1) # for ibmqx2
    #fSWAP(Udis,q1,q2) # for ibmqx4
    circuit += F0(q0,q2)
    circuit += F0(q1,q3)
    #fSWAP(Udis,q2,q1) # for ibmqx2
    #fSWAP(Udis,q1,q2) # for ibmqx4
    return circuit

def Ising(lam,q0,q1,q2,q3):
    if lam<1:
        return tq.gates.X(q3) + Udisg(lam,q0,q1,q2,q3)
    else:
        return Udisg(lam,q0,q1,q2,q3)
    

In [17]:
def digit_sum(n):
    num_str = str(n)
    sum = 0
    for i in range(0, len(num_str)):
        sum += int(num_str[i])
    return sum

In [20]:
#simulation
def sim_run_expecM(lam):
    shots = 1024

    H = tq.paulis.Z([1])

    circuit = Ising(lam,0,1,2,3)

    M_exp = tq.ExpectationValue(H = H, U = circuit)
    M=tq.simulate(M_exp,backend='qiskit',samples=shots)
    return M

# Real Device
def real_device_run_expecM(real_device,lam):
    shots = 1024
    H = tq.paulis.Z([1])
    circuit = Ising(lam,0,1,2,3)

    M_exp = tq.ExpectationValue(H = H, U = circuit)
    M=tq.simulate(M_exp,backend='qiskit',device = real_device,samples=shots)
    return M

# Real Device but only simulate wavefunction
def real_device_run_sumM(real_device,lam):
    shots = 1024
    mag_real_res = []
    wfn_real = tq.simulate(Ising(lam,0,1,2,3),backend='qiskit',device = real_device,samples=shots)
    r1_real = []
    r2_real = []
    for key, value in wfn_real.items():
        r1_real +=[key.binary]
        r2_real +=[value]
    M_real = 0
    for j in range(len(r1_real)):
        M_real = M_real + (4-2*digit_sum(r1_real[j]))*r2_real[j]/shots
    
    return M_real/4

In [9]:
real_device_run_expecM('ibmq_vigo',0.2)

1.0

In [11]:
sim_run_expecM(0.2)

0.08203125

In [21]:
real_device_run_sumM('ibmq_vigo',0.2)

0.06494140625